# KLH  Phylogeny

In [1]:
# Author: Dean Hoffer
# This script grabs the protein sequence for KLH1 and KLH2 from uniprot,
# blasts the protein sequence,
# and returns protein IDs for proteins with an e-value of less than 1e-7.
# Then the fasta files are pulled from Entrez, 
# multiple sequence analysis is done by Clustalw2,
# and a phylogenetic tree is created

In [3]:
import requests
import subprocess
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import Entrez, SeqIO, AlignIO, Phylo
from Bio.Align.Applications import ClustalwCommandline
Entrez.email = "hofferd78@gmail.com"

In [4]:
def write_fasta(filename, header, sequence):
    """Function to write fasta files
    filename: string, should end in ".fasta"
    header: string, sequence header
    sequence: string, sequence content
    
    >>>write_fasta("KLH1.fasta", KLH1_header, KLH1_seq)
    """
    f = open(filename, "a")
    f.write(header)
    for line in sequence:
        f.write("\n")
        f.write(line.strip("'"))
    f.close()

## Blast KLH1 and KLH2

### Get KLH1 and KLH2 protein sequences from uniprot

In [5]:
# get protein sequence using KLH1 and KLH2 uniprot protein ID
KLH1 = requests.get("https://www.uniprot.org/uniprot/Q10583.fasta")
KLH2 = requests.get("https://www.uniprot.org/uniprot/Q10584.fasta")

In [6]:
# wrangle data into fasta format
KLH1_resp = str(KLH1.content).split('\\n')
KLH2_resp = str(KLH2.content).split('\\n')

KLH1_header = KLH1_resp[0][2:]
KLH1_seq = KLH1_resp[1:]

KLH2_header = KLH2_resp[0][2:]
KLH2_seq = KLH2_resp[1:]

In [10]:
# create empty files
KLH1_fasta = "KLH1.faa"
KLH2_fasta = "KLH2.faa"
f1 = open(KLH1_fasta, "w")
f2 = open(KLH2_fasta, "w")

In [11]:
# write KLH1.faa and KLH2.faa 
write_fasta(KLH1_fasta, KLH1_header, KLH1_seq)
f1.close()
write_fasta(KLH2_fasta, KLH2_header, KLH2_seq)
f2.close()

### Blastp KLH1 and KLH2 protein sequences to swiss prot database

In [12]:
# protein BLAST KLH1 fasta to swissprot database over the internet and save results to XML file
KLH1_string = open(KLH1_fasta).read()
KLH1_result_handle = NCBIWWW.qblast("blastp", "swissprot", KLH1_string)
with open("KLH1_swissprot.xml", "w") as out_handle:
    out_handle.write(KLH1_result_handle.read())
KLH1_result_handle.close()

In [13]:
# protein BLAST KLH2 fasta to swissprot database over the internet and save results to XML file
KLH2_string = open(KLH2_fasta).read()
KLH2_result_handle = NCBIWWW.qblast("blastp", "swissprot", KLH2_string)
with open("KLH2_swissprot.xml", "w") as out_handle:
    out_handle.write(KLH2_result_handle.read())
KLH2_result_handle.close()

### Parse blast results for top hits

In [14]:
# open and read XML blast result files
KLH1_handle = open("KLH1_swissprot.xml")
KLH2_handle = open("KLH2_swissprot.xml")
KLH1_blast_record = NCBIXML.read(KLH1_handle)
KLH2_blast_record = NCBIXML.read(KLH2_handle)

In [17]:
# parse blast results to create list of protein IDs with a blast E value lower than 1e-7
KLH1_blast_hits = []
E_VALUE_THRESH = 0.0000001
for alignment in KLH1_blast_record.alignments:
#     print("***ALIGNMENT***")
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            for entry in str(alignment.title).split(">"):
                ID = entry.split("|")[1]
                if ID not in KLH1_blast_hits:
                    KLH1_blast_hits.append(ID)
#                 print(entry.split("|")[1])
#             print(str(alignment.title))
            break

In [18]:
# parse blast results to create list of protein IDs with a blast E value lower than 1e-7
KLH2_blast_hits = []
for alignment in KLH2_blast_record.alignments:
#     print("***ALIGNMENT***")
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            for entry in str(alignment.title).split(">"):
                ID = entry.split("|")[1]
                if ID not in KLH2_blast_hits:
                    KLH2_blast_hits.append(ID)           
#                 print(entry.split("|")[1])
#             print(str(alignment.title))
            break

## Multisequence Alignment

### Get sequences for top hits

In [21]:
# get protein sequences from Entrez and write to fasta file
with open("KLH1_blast_results.faa", "w") as out_handle:
    for p_id in KLH1_blast_hits:
        fasta = Entrez.efetch(db = "protein", id = p_id, rettype = "fasta")
        fasta_record = SeqIO.read(fasta, "fasta")
        out_handle.write(f'>{fasta_record.id}|{fasta_record.description}\n{fasta_record.seq}\n')

In [22]:
# get protein sequences from Entrez and write to fasta file
with open("KLH2_blast_results.faa", "w") as out_handle:
    for p_id in KLH2_blast_hits:
        fasta = Entrez.efetch(db = "protein", id = p_id, rettype = "fasta")
        fasta_record = SeqIO.read(fasta, "fasta")
        out_handle.write(f'>{fasta_record.id}|{fasta_record.description}\n{fasta_record.seq}\n')

### Clustalw2 MSA

In [23]:
# perform multisequence alignment with blast results fasta files
# alignment files can be uploaded to EMBL-EBI ClustalW2 phylogeny tool located at:
# http://www.ebi.ac.uk/Tools/services/web/toolform.ebi?tool=clustalw2_phylogeny
cline = ClustalwCommandline("clustalw2", infile = "KLH1_blast_results.faa")
stdout, stderr = cline()
KLH1_msa_align = AlignIO.read("KLH1_blast_results.aln", "clustal")

cline = ClustalwCommandline("clustalw2", infile = "KLH2_blast_results.faa")
stdout, stderr = cline()
KLH2_msa_align = AlignIO.read("KLH2_blast_results.aln", "clustal")

### Create Phylogenetic Tree

In [24]:
# create and visualize phylogenetic tree for KLH1 multiple sequence alignment
KLH1_tree = Phylo.read("KLH1_blast_results.dnd", "newick")
Phylo.draw_ascii(KLH1_tree)

   _______ sp|Q10583.2|HCY1_MEGCR|sp|Q10583.2|HC...
 ,|
 ||_____________ sp|P12031.2|HCYB_HELPO|sp|P12031.2|HC...
 |
 |            , sp|O61363.1|HCYG_ENTDO|sp|O61363.1|HC...
 |        ____|
 |     __|    |_ sp|P12659.2|HCYA_ENTDO|sp|P12659.2|HC...
 |    |  |
 |   ,|  |________ sp|P56826.1|HCYG_SEPOF|sp|P56826.1|HC...
 |   ||
 |  ,||___________________ sp|P56824.1|HCYC_SEPOF|sp|P56824.1|HC...
 |  ||
 |  ||________________ sp|P56825.1|HCYE_SEPOF|sp|P56825.1|HC...
 |  |
 |  |                       __ sp|P06845.3|TYRO_STRGA|sp|P06845.3|TY...
 |  |                     ,|
 |  |                    ,||__ sp|P07524.2|TYRO_STRAT|sp|P07524.2|TY...
 |  |                    ||
 |  |              ______||____ sp|P55023.2|TYRO_STRLN|sp|P55023.2|TY...
 |  |             |      |
 |  |        _____|      |_____ sp|P55022.2|TYRO_STRGB|sp|P55022.2|TY...
 |  |       |     |
 | _|      ,|     |_____________ sp|B1VTI5.1|GRIF_STRGG|sp|B1VTI5.1|GR...
 || |      ||
 || |      ||      ________________ sp|Q19673.

In [25]:
# create and visualize phylogenetic tree for KLH2 multiple sequence alignment
KLH2_tree = Phylo.read("KLH2_blast_results.dnd", "newick")
Phylo.draw_ascii(KLH2_tree)

                       , sp|Q10584.2|HCY2_MEGCR|sp|Q10584.2|HC...
             __________|
            |          |_____ sp|P81732.1|HC2C_MEGCR|sp|P81732.1|HC...
            |
            |   _____________ sp|P56823.1|HCYG_HELPO|sp|P56823.1|HC...
            |__|
           _|  |______________ sp|P80960.2|HCY2A_RAPVE|sp|P80960.2|H...
          | |
          | | __________ sp|Q10583.2|HCY1_MEGCR|sp|Q10583.2|HC...
        __| ||
       |  |  |_______________ sp|P12031.2|HCYB_HELPO|sp|P12031.2|HC...
       |  |
       |  |__________________ sp|P83040.1|HCY2E_RAPVE|sp|P83040.1|H...
       |
  _____|                , sp|O61363.1|HCYG_ENTDO|sp|O61363.1|HC...
 |     |          ______|
 |     |     ____|      |_ sp|P12659.2|HCYA_ENTDO|sp|P12659.2|HC...
 |     |    |    |
 |     |  __|    |___________ sp|P56826.1|HCYG_SEPOF|sp|P56826.1|HC...
 |     | |  |
 |     |_|  |______________________ sp|P56824.1|HCYC_SEPOF|sp|P56824.1|HC...
 |       |
 |       |____________________ sp|P56825.1|HCYE_SEPOF